<a href="https://colab.research.google.com/github/sasikiranaratla/qlora-tone-adapter/blob/main/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Install Unsloth & Dependencies
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes wandb datasets

# 2. Login to Hugging Face and Weights & Biases
from huggingface_hub import login
from google.colab import userdata
import wandb

# Use Colab Secrets (the 🔑 icon) to store your tokens as 'HF_TOKEN' and 'WANDB_API_KEY'
login(userdata.get('HF_TOKEN'))
wandb.login(key=userdata.get('WANDB_API_KEY'))

# Initialize W&B project
wandb.init(project="llama-3.2-counselor", job_type="training")

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-zs7n_4bo/unsloth_adc42a33f87e4ffe84570d6a67ca7856
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-zs7n_4bo/unsloth_adc42a33f87e4ffe84570d6a67ca7856
  Resolved https://github.com/unslothai/unsloth.git to commit 8ea5338154859ed25b50366cb1264ed4d933eae3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.7/290.7 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 106.6 MB/s eta 0:00:00

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sasikiranaratla (sasikiranaratla-sasikiranaratla) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
from datasets import load_dataset

# 1. Load the local file from Colab storage
# Make sure 'counsellor_data.jsonl' is uploaded to the folder icon on the left
local_file = "counsellor_data.jsonl"
dataset = load_dataset("json", data_files=local_file, split="train")

# 2. Push to Hugging Face Hub
# This creates a repository and uploads the data in Parquet format (optimized for LLMs)
dataset_repo = "sasikiranaratla/proverbial-counselor-data"
dataset.push_to_hub(dataset_repo)

print(f"Dataset successfully uploaded to: https://huggingface.co/datasets/{dataset_repo}")



Generating train split: 0 examples [00:00, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              : 100%|##########| 52.3kB / 52.3kB            

                              : 100%|##########| 52.3kB / 52.3kB            

Dataset successfully uploaded to: https://huggingface.co/datasets/sasikiranaratla/proverbial-counselor-data


In [3]:
# (Optional) Reload to verify
from datasets import load_dataset
dataset_repo = "sasikiranaratla/proverbial-counselor-data"
dataset = load_dataset(dataset_repo, split="train")

README.md:   0%|          | 0.00/342 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/52.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/301 [00:00<?, ? examples/s]

In [9]:
from transformers import AutoTokenizer
import matplotlib.pyplot as plt

# Load your specific tokenizer
model_id = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit" # Example: replace with your model
tokenizer = AutoTokenizer.from_pretrained(model_id)

dataset_split = dataset.train_test_split(test_size=0.1, seed=3407)

train_ds = dataset_split["train"]
eval_ds = dataset_split["test"]

# Quick check of your data lengths
lengths = [len(tokenizer.encode(str(msg))) for msg in dataset["messages"]]
print(f"Longest example: {max(lengths)} tokens")
print(f"Average example: {sum(lengths)/len(lengths):.0f} tokens")

Longest example: 170 tokens
Average example: 145 tokens


In [10]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length = 512,
    load_in_4bit = True,
)



/tmp/ipython-input-4036208777.py:1: UserWarning: WARNING: Unsloth should be imported before [transformers] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [11]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Ll

In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
)

Unsloth 2025.12.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [16]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Apply chat template to dataset
def formatting_prompts_func(examples):
    texts = [tokenizer.apply_chat_template(msg, tokenize=False, add_generation_prompt=False) for msg in examples["messages"]]
    return { "text" : texts }

train = train_ds.map(formatting_prompts_func, batched = True)
val = eval_ds.map(formatting_prompts_func, batched = True)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train,
    eval_dataset = val,
    dataset_text_field = "text",
    max_seq_length = 512,
    args = TrainingArguments(
        # --- BATCHING & SPEED ---
        per_device_train_batch_size = 4,      # Optimized for 512 seq_len
        gradient_accumulation_steps = 4,     # Global Batch Size = 16
        warmup_ratio = 0.1,                  # 10% of total steps for warmup
        num_train_epochs = 2,                # 3 full passes of your 300+ samples
        max_steps = -1,                      # Let epochs decide the length
        learning_rate = 2e-4,                # Ideal for LoRA/Proverb training
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        group_by_length = True,              # Faster training (packs similar lengths)

        # --- OPTIMIZATION ---
        optim = "adamw_8bit",                # Lower VRAM footprint
        weight_decay = 0.01,
        logging_steps = 1,
        output_dir = "outputs",

        # --- EVALUATION & SAVING ---
        eval_strategy = "steps",
        eval_steps = 20,                     # Frequent checks for 113 total steps
        save_strategy = "steps",
        save_steps = 20,                     # Backup every 20 steps
        save_total_limit = 2,                # Keep only latest 2 to save Colab disk space
        per_device_eval_batch_size = 1,      # Prevents OOM during validation

        # --- HUB & W&B SETTINGS ---
        report_to = "wandb",
        run_name = "llama3-2-3b-counselor-v1",
        push_to_hub = True,
        hub_model_id = "sasikiranaratla/llama-3.2-3b-counselor",
        hub_strategy = "checkpoint",         # Ensures checkpoints are pushed to HF
        hub_private_repo = True,             # Keep your work private
    ),
)



Map:   0%|          | 0/270 [00:00<?, ? examples/s]

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=1):   0%|          | 0/270 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=1):   0%|          | 0/31 [00:00<?, ? examples/s]

In [15]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 270 | Num Epochs = 3 | Total steps = 51
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
20,1.022500,1.114170
40,0.799200,0.996643


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


eval/loss,█▁
eval/runtime,█▁
eval/samples_per_second,▁█
eval/steps_per_second,▁█
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,██▇▅▄▄▃▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▂▁▂▁▂▁▂
train/learning_rate,▁▂▃▅▆████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▁▁
train/loss,▇██▇▆▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁
eval/loss,0.99664
eval/runtime,11.3427


TrainOutput(global_step=51, training_loss=1.3096125558310865, metrics={'train_runtime': 356.5561, 'train_samples_per_second': 2.272, 'train_steps_per_second': 0.143, 'total_flos': 2024896454295552.0, 'train_loss': 1.3096125558310865, 'epoch': 3.0})

In [17]:
# Push final LoRA adapters
model.push_to_hub("sasikiranaratla/llama-3.2-3b-counselor", token = userdata.get('HF_TOKEN'))
#tokenizer.push_to_hub("sasikiranaratla/llama-3.2-3b-counselor", token = userdata.get('HF_TOKEN'))

# Optional: Save as GGUF for local use (Ollama)
#model.push_to_hub_gguf("sasikiranaratla/llama-3.2-3b-counselor-GGUF", tokenizer, quantization_method = "q4_k_m")



README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:   9%|8         | 8.28MB / 97.3MB            

No files have been modified since last commit. Skipping to prevent empty commit.


Saved model to https://huggingface.co/sasikiranaratla/llama-3.2-3b-counselor


In [18]:
wandb.finish()

In [21]:
from unsloth import FastLanguageModel

# 1. Enable 2x faster inference
FastLanguageModel.for_inference(model)

# 2. Define your test message
# Note: Keep the exact same system prompt you used in your training data!
messages = [
    {"role": "system", "content": "You are a wise mental health counselor. Every response must begin with a relevant proverb or ancient saying, followed by empathetic validation and a clinical discovery question."},
    {"role": "user", "content": "I feel like I'm constantly racing against time, but I never get anything done"}
]

# 3. Apply the chat template
# CRITICAL: add_generation_prompt must be True here
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # This adds the <|start_header_id|>assistant<|end_header_id|> tag
    return_tensors = "pt",
).to("cuda")

In [22]:
from transformers import TextStreamer

# Define the streamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)

# Generate the response
_ = model.generate(
    input_ids = inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
    temperature = 0.7,
    min_p = 0.1
)

"Time is a river that moves both swift and slow." It is easy to get caught up in the speed of the river and forget that the current is always moving, whether you are moving or not. When you finally stop trying to keep up with the clock, what is the first thing you would do with your time?<|eot_id|>
